# Data pipeline for our mobile application "Path to Profession"

### Selenium and BeautifulSoup work together like a team to collect and clean data from Naukri.com, making it easier to use for analysis and decision-making. They're like a dynamic duo for web data.

#### Initializing selenium 

In [1]:
from datetime import date
# impoting requests module  to send HTTP requests using Python
import requests
from bs4 import BeautifulSoup
import pandas as pd
#re stands for regular expression which specifies a set of strings that matches it.
import re
import time
#Display a Python object in all frontends. 
from IPython.display import clear_output
#random module in Python defines a series of functions for generating or manipulating random integers. 
import random
#most common exceptions in Selenium WebDriver, and it is thrown when an HTML element cannot be found.
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
#ChromeOptions() for manipulating various properties of the Chrome driver
options = webdriver.ChromeOptions()
#to avoid the situation if the security code is outdated and in result users will not be allowed to access that website
options.add_argument('--ignore-certificate-errors')
#we added icognito to prevent the popping of login page
options.add_argument('--incognito')
#to browser open with UI
options.add_argument('--headless')
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(executable_path='chromedriver.exe')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17952\542939457.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe')


Removing all the spaces from the string 

In [69]:
list_jobs=["Cloud Solution Architect","Software Architect"]
list1=[]
today =str(date.today())
for i in list_jobs:
      for j in i:
        if j==" ":
            i=i.replace(" ","-")
            list1.append(i)
myFinalList = pd.unique(list1).tolist()
list1=myFinalList

automating web search and scraping data from naukri .com

In [65]:
linkstart="https://www.naukri.com/"
for x in range(0,2):
    Link=[]
    for page in range(0,1): #first two pages of the naukri.com search
        driver.get(linkstart+list1[x]+"-jobs-in-bangalore-"+str(page))
        soup=BeautifulSoup(driver.page_source, 'lxml')#returns html of the page
        time.sleep(3)
        container = driver.find_element(By.ID, 'listContainer')
        anchors = container.find_elements(By.CLASS_NAME, 'title')
        for anchor in anchors:
            if anchor.get_attribute('href'):
                Link.append(anchor.get_attribute('href'))
    print(len(Link)) #total link
    #these were all the columns we got from naukri.com and made empty list 
    salary=[]
    experience=[]
    Location=[]
    description=[]
    role=[]
    industry_type=[]
    qualification=[]
    Functional_area=[]
    Employment_type=[]
    Role_category=[]
    company=[]
    skills=[]
    for lin in range(len(Link)):
        driver.get(Link[lin])
        time.sleep(2)
        soup=BeautifulSoup(driver.page_source, 'lxml') 
        if soup.find(attrs={'class':"salary"})==None: #to skip the error
            continue
        else:
            experience.append(soup.find(attrs={'class':"exp"}).text)
            salary.append(soup.find(attrs={'class':"salary"}).text)
            Location.append(soup.find(attrs={'class':'loc'}).find('a').text)
            description.append(soup.find(attrs={'class':"job-desc"}).text)
            details=[]
            for i in soup.find(attrs={'class':"other-details"}).findAll(attrs={'class':"details"}):
                details.append(i.text)
            role.append(details[0])
            industry_type.append(details[1])
            Functional_area.append(details[2])
            Employment_type.append(details[3])
            Role_category.append(details[4])
            qual=[]
            for i in soup.find(attrs={'class':"education"}).findAll(attrs={'class':'details'}):
                qual.append(i.text)
            qualification.append(qual)
            sk=[]
            for i in soup.find(attrs={'class':"key-skill"}).findAll('a'):
                sk.append(i.text)
            skills.append(sk)
            company.append(soup.find(attrs={'class':"about-company"}).find(attrs={'class':"detail dang-inner-html"}).text)
    df=pd.DataFrame()
    df['company']=company
    df['role']=role
    df['salary']=salary
    df['experience']=experience
    df['Location']=Location
    df['description']=description
    df['skills']=skills
    df['qualification']=qualification
    df['industry_type']=industry_type
    df['Functional_area']=Functional_area
    df['Employment_type']=Employment_type
    df['Role_category']=Role_category
    len(df)
    df.head()
    df.to_csv(list1[x]+'.csv',index=False)
    Link.clear()
print(df)                

20
20
Empty DataFrame
Columns: [company, role, salary, experience, Location, description, skills, qualification, industry_type, Functional_area, Employment_type, Role_category]
Index: []


In [2]:
driver.get('https://www.naukri.com/software-architect-jobs-in-pune')
time.sleep(3)
data = {
    "experience": [],
    "salary": [],
    "location":[],
    "Job_Description": [],
    "Role":[],
    "Industry_Type": [],
    "Department": [],
    "Employment_Type": [],
    "Role_Category": [],
    "Skills": []
}
container = driver.find_element(By.ID, 'listContainer')
wrappers = container.find_elements(By.CLASS_NAME, "srp-jobtuple-wrapper")
for wrap in wrappers:
    data['experience'].append(wrap.find_element(By.CLASS_NAME, "exp").text)
    data['salary'].append(wrap.find_element(By.CLASS_NAME, "sal").text) 
    data['location'].append(wrap.find_element(By.CLASS_NAME, "loc").text)
    link = wrap.find_element(By.CLASS_NAME, 'title').get_attribute('href')
    tab = webdriver.Chrome(executable_path='chromedriver.exe')
    tab.get(link)
    time.sleep(2)
    section = tab.find_elements(By.TAG_NAME, "section")
    divisions = section[2].find_elements(By.XPATH, './div')
    data['Job_Description'].append(divisions[0].text)
    division2 = divisions[1].text.split('\n')
    
    data['Role'].append(division2[0].split(":")[1])
    data['Industry_Type'].append(division2[1].split(":")[1])
    data['Employment_Type'].append(division2[2].split(":")[1])
    data['Department'].append(division2[3].split(":")[1])
    data['Role_Category'].append(division2[4].split(":")[1])
    skills = divisions[3].find_elements(By.TAG_NAME, 'span')
    skills2 = ''
    for skill in skills:
        skills2 += skill.text +', ' 
    data['Skills'].append(skills2.rstrip())
    print(skills2.rstrip())
    tab.close()    

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17952\2470972817.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  tab = webdriver.Chrome(executable_path='chromedriver.exe')


Rte, BSW, MCAL Drives, AUTOSAR, Adc, Pwm, Dio, mcal, uml, Ecu, Rhapsody, Aspice, sdlc,
Software Architecting, Jenkins, framework, GIT, data visualization, JIRA, MATLAB, Python, ASPICE,
Linux, Python,
Design, python, software development,
Architecture, Agile development, Javascript, Software Architect, Scrum, Subject Matter Expert, Customer engagement, Advertising,
Computer science, Front end, Enterprise applications, TDD, Agile, Healthcare, Product design, Test cases, cisco, Information technology,
AUTOSAR, System architecture, Software design, UML, Staffing, Architectural design, Tool design, Software Architect, Hardware, Embedded software,
Supply chain, Career development, Operations research, Mechatronics, Leadership development, Warehouse, Simulation, Agile, Industrial engineering, Python,
Automotive Embedded Systems, linux, qnx, Embedded Development, Embedded Software Development, Embedded Software,
Computer science, Automation, NoSQL, Web development, Javascript, Entity framework

In [7]:
df = pd.DataFrame(data)
df.to_csv("xyz.csv")

In [159]:
data

{'experience': ['8-12 Yrs', '8-10 Yrs', '9-14 Yrs', '8-12 Yrs', '10-14 Yrs'],
 'salary': ['15-25 Lacs PA',
  'Not disclosed',
  'Not disclosed',
  'Not disclosed',
  'Not disclosed'],
 'location': ['Hybrid - Pune',
  'Pune, Maharashtra',
  'Hybrid - Pune',
  'Pune',
  'Pune'],
 'Job_Description': ['Hello,\n\nAs part of our new growth strategy at BHTC India, we are looking to increase our internal Application and Project Team.\nWe are looking for experienced who have hands on experience AUTOSAR architecture and concepts - BSW, RTE and SWC.\nExperience on MCAL drivers(DIO/ADC/I2C/EEPROM/PWM/CAN/LIN etc.),System level knowledge\nASPICE experience, Automotive SDLC experience, Knowledge of Rhapsody and UML will be an added advantage.\nThe position would be based at MIDC, Bhosari Pune.\nSome of the benefits include - 10 days work from home per month\nFree access to LinkedIn Learning\nCanteen Facility that takes care of breakfast, lunch and snacks\nWorkshops like customer engagement, German L

In [163]:
driver.get("https://www.naukri.com/job-listings-software-architect-akal-information-systems-ltd-bangalore-bengaluru-10-to-14-years-210923909577?src=jobsearchDesk&sid=16953296606939337&xp=1&px=1")


In [164]:
section = driver.find_elements(By.TAG_NAME, "section")

In [165]:
divisions = section[2].find_elements(By.TAG_NAME, 'div')

In [154]:
skills = divisions[12].find_elements(By.TAG_NAME, 'span')

In [171]:
division2 = section[2].find_elements(By.XPATH, './div')

In [180]:
skills = division2[3].find_elements(By.TAG_NAME, 'span')

In [181]:
for skill in skills:
    print(skill.text)

Java
Hibernate
software development
SOA
Mule
Software Architecture
Java Architect
software engineering methodologies
Spring
Microservices
Data Structure
Sdlc
Architect
Algorithms
RMQ
j2ee
Agile
Spring boot
Aws
